In [3]:
#install Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [10]:
#Check versions
print(f"pandas: {pd.__version__}")
print(f"Numpy: {np.__version__}")
print("✅ Environment ready!")

pandas: 2.3.2
Numpy: 1.26.4
✅ Environment ready!


In [31]:
# Set display options for better readability
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', None)


print("="*70)
print("TELECOM CUSTOMER CHURN ANALYSIS - INITIAL EXPLORATION")
print("="*70)

TELECOM CUSTOMER CHURN ANALYSIS - INITIAL EXPLORATION


In [22]:
# ============================================================================
# STEP 1: LOAD THE DATA
# ============================================================================

df = pd.read_csv(r'C:\Users\kramo\OneDrive\Documents\Desktop\WAHID\PORTFOLIO\Analytics Project\Customer Churn Analytics\Telco-Customer-Churn.csv')

print("\n✅ Data loaded successfully!")
print(f"Dataset shape: {df.shape[0]:,} customers × {df.shape[1]} features\n")


✅ Data loaded successfully!
Dataset shape: 7,043 customers × 21 features



In [32]:
# ============================================================================
# STEP 2: FIRST LOOK AT THE DATA
# ============================================================================
print("="*70)
print("FIRST 5 CUSTOMERS")
print("="*70)
print(df.head)

FIRST 5 CUSTOMERS
<bound method NDFrame.head of       customerID  gender  SeniorCitizen Partner Dependents  tenure  \
0     7590-VHVEG  Female              0     Yes         No       1   
1     5575-GNVDE    Male              0      No         No      34   
2     3668-QPYBK    Male              0      No         No       2   
3     7795-CFOCW    Male              0      No         No      45   
4     9237-HQITU  Female              0      No         No       2   
...          ...     ...            ...     ...        ...     ...   
7038  6840-RESVB    Male              0     Yes        Yes      24   
7039  2234-XADUH  Female              0     Yes        Yes      72   
7040  4801-JZAZL  Female              0     Yes        Yes      11   
7041  8361-LTMKD    Male              1     Yes         No       4   
7042  3186-AJIEK    Male              0      No         No      66   

     PhoneService     MultipleLines InternetService OnlineSecurity  \
0              No  No phone service      

In [28]:
# ============================================================================
# STEP 3: COLUMN INFORMATION
# ============================================================================

print("\n" + "="*70)
print("DATASET STRUCTURE")
print("="*70)
print(f"\nTotal columns: {len(df.columns)}")
print("\nColumn names and types:")
print(df.dtypes)


DATASET STRUCTURE

Total columns: 21

Column names and types:
customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object


In [36]:
# ============================================================================
# STEP 4: TARGET VARIABLE ANALYSIS (CHURN)
# ============================================================================
print("\n" + "="*70)
print("🎯 TARGET VARIABLE: CHURN")
print("="*70)


churn_counts = df['Churn'].value_counts()
churn_pct = df['Churn'].value_counts(normalize=True) * 100

print(f"\nChurn Distribution:")
print(f"  No  (Retained):  {churn_counts['No']:,} customers ({churn_pct['No']:.1f}%)")
print(f"  Yes (Churned):   {churn_counts['Yes']:,} customers ({churn_pct['Yes']:.1f}%)")

# Calculate baseline churn rate
baseline_churn_rate = churn_pct['Yes']
print(f"\n📊 BASELINE CHURN RATE: {baseline_churn_rate:.2f}%")
print("   (This is what we're trying to reduce!)")



🎯 TARGET VARIABLE: CHURN

Churn Distribution:
  No  (Retained):  5,174 customers (73.5%)
  Yes (Churned):   1,869 customers (26.5%)

📊 BASELINE CHURN RATE: 26.54%
   (This is what we're trying to reduce!)


In [38]:
# ============================================================================
# STEP 5: MISSING VALUES CHECK
# ============================================================================
print("\n" + "="*70)
print("MISSING VALUES ANALYSIS")
print("="*70)

missing_values = df.isnull().sum()
missing_pct = (df.isnull().sum() / len(df)) * 100

missing_df = pd.DataFrame({
    'Missing_Count': missing_values,
    'Percentage': missing_pct
})
missing_df = missing_df[missing_df['Missing_Count'] > 0].sort_values('Missing_Count', ascending=False)

if len(missing_df) > 0:
    print("\n⚠️  Columns with missing values:")
    print(missing_df)
else:
    print("\n✅ No missing values detected!")


MISSING VALUES ANALYSIS

✅ No missing values detected!


In [63]:
# ============================================================================
# STEP 6: DATA TYPE ISSUES
# ============================================================================
print("\n" + "="*70)
print("🚨 DATA QUALITY CHECKS")
print("="*70)

# Check TotalCharges - should be numeric but might be stored as string
print(f"\nTotalCharges data type: {df['TotalCharges'].dtype}")

if df['TotalCharges'].dtype == 'object':
    print("⚠️  WARNING: TotalCharges is stored as string (should be numeric)")
    print("   This will need to be fixed in data cleaning!")

# Check for non-numeric values
non_numeric = df[pd.to_numeric(df['TotalCharges'], errors='coerce').isnull()]['TotalCharges']
print(non_numeric.shape)
print(f"   Found {len(non_numeric)} non-numeric values")


if len(non_numeric) > 0:
        print(f"   Examples: {non_numeric.head().tolist()}")
   



🚨 DATA QUALITY CHECKS

TotalCharges data type: object
⚠️  WARNING: TotalCharges is stored as string (should be numeric)
   This will need to be fixed in data cleaning!
(11,)
   Found 11 non-numeric values
   Examples: [' ', ' ', ' ', ' ', ' ']


In [64]:
# ============================================================================
# STEP 7: BASIC STATISTICS
# ============================================================================
print("\n" + "="*70)
print("NUMERICAL FEATURES - SUMMARY STATISTICS")
print("="*70)
print(df.describe())


NUMERICAL FEATURES - SUMMARY STATISTICS
       SeniorCitizen       tenure  MonthlyCharges
count    7043.000000  7043.000000     7043.000000
mean        0.162147    32.371149       64.761692
std         0.368612    24.559481       30.090047
min         0.000000     0.000000       18.250000
25%         0.000000     9.000000       35.500000
50%         0.000000    29.000000       70.350000
75%         0.000000    55.000000       89.850000
max         1.000000    72.000000      118.750000


In [66]:
# ============================================================================
# STEP 8: CATEGORICAL FEATURES OVERVIEW
# ============================================================================
print("\n" + "="*70)
print("CATEGORICAL FEATURES - UNIQUE VALUES")
print("="*70)

categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    unique_count = df[col].nunique()
    print(f"\n{col}:")
    print(f"  Unique values: {unique_count}")
    if unique_count <= 10:  # Only show if not too many values
        print(f"  Values: {df[col].unique().tolist()}")


CATEGORICAL FEATURES - UNIQUE VALUES

customerID:
  Unique values: 7043

gender:
  Unique values: 2
  Values: ['Female', 'Male']

Partner:
  Unique values: 2
  Values: ['Yes', 'No']

Dependents:
  Unique values: 2
  Values: ['No', 'Yes']

PhoneService:
  Unique values: 2
  Values: ['No', 'Yes']

MultipleLines:
  Unique values: 3
  Values: ['No phone service', 'No', 'Yes']

InternetService:
  Unique values: 3
  Values: ['DSL', 'Fiber optic', 'No']

OnlineSecurity:
  Unique values: 3
  Values: ['No', 'Yes', 'No internet service']

OnlineBackup:
  Unique values: 3
  Values: ['Yes', 'No', 'No internet service']

DeviceProtection:
  Unique values: 3
  Values: ['No', 'Yes', 'No internet service']

TechSupport:
  Unique values: 3
  Values: ['No', 'Yes', 'No internet service']

StreamingTV:
  Unique values: 3
  Values: ['No', 'Yes', 'No internet service']

StreamingMovies:
  Unique values: 3
  Values: ['No', 'Yes', 'No internet service']

Contract:
  Unique values: 3
  Values: ['Month-to-mont

In [68]:
# ============================================================================
# STEP 9: KEY INSIGHTS - INITIAL OBSERVATIONS
# ============================================================================
print("\n" + "="*70)
print("🔍 KEY INITIAL OBSERVATIONS")
print("="*70)

print(f"""
1. DATASET SIZE:
   - {df.shape[0]:,} total customers
   - {df.shape[1]} features to analyze
   
2. CHURN RATE:
   - {baseline_churn_rate:.1f}% of customers have churned
   - This is our baseline to improve upon
   
3. DATA QUALITY:
   - TotalCharges needs type conversion
   - Need to investigate missing/empty values
   
4. NEXT STEPS:
   - Clean TotalCharges column
   - Handle any missing values
   - Begin exploratory analysis of churn patterns
""")



🔍 KEY INITIAL OBSERVATIONS

1. DATASET SIZE:
   - 7,043 total customers
   - 21 features to analyze
   
2. CHURN RATE:
   - 26.5% of customers have churned
   - This is our baseline to improve upon
   
3. DATA QUALITY:
   - TotalCharges needs type conversion
   - Need to investigate missing/empty values
   
4. NEXT STEPS:
   - Clean TotalCharges column
   - Handle any missing values
   - Begin exploratory analysis of churn patterns



In [69]:
# ============================================================================
# SAVE THIS INITIAL SUMMARY
# ============================================================================
print("\n" + "="*70)
print("✅ INITIAL EXPLORATION COMPLETE!")
print("="*70)
print("\nWhat you've accomplished:")
print("  ✓ Loaded dataset successfully")
print("  ✓ Identified target variable (Churn)")
print("  ✓ Calculated baseline churn rate")
print("  ✓ Discovered data quality issues")
print("  ✓ Understood dataset structure")
print("\n*📝 Document these findings in your README!")
print("🎯 Next session: Data Cleaning")


✅ INITIAL EXPLORATION COMPLETE!

What you've accomplished:
  ✓ Loaded dataset successfully
  ✓ Identified target variable (Churn)
  ✓ Calculated baseline churn rate
  ✓ Discovered data quality issues
  ✓ Understood dataset structure

*📝 Document these findings in your README!
🎯 Next session: Data Cleaning
